In [129]:
import requests
import pandas as pd

In [37]:
!aws s3 ls --recursive s3://snowex-data/uavsar-project/UAVSAR_images/ > files.txt

In [35]:
folders = []
with open('folders.txt', 'r') as folder_file:
    lines = folder_file.readlines()
    for l in lines:
        folders.append(l.split(' ')[-1][:-2])

In [40]:
files = []
with open('files.txt', 'r') as files_file:
    lines = files_file.readlines()
    for l in lines:
        path = l.split(' ')[-1][:-1]
        f = path.split('/')[-1].split('.')[0]
        files.append(f)

In [50]:
# Now, we want to remove duplicates. To do that we can generate a dict with keys from the list
# Dictionnaries don't accept duplicates, and will automatically trim the list for us
# Then convert back to a list
dico = dict.fromkeys(files)
granule_names = list(dico.keys())
granule_names.remove('test')
granule_names = ['UA_' + g for g in granule_names]

In [116]:
#Getting read of the polarization info
granule_names_clean = []
for granule_name in granule_names:
    els = granule_name.split('_')
    els[-2] = els[-2][:-2]
    new_name = '_'.join(els)
    granule_names_clean.append(new_name)
#granule_names[0].split('_')[-2][:-2]

In [118]:
# Now, we want to remove duplicates AGAIN (because of polarizations names that are dropped now).
dico = dict.fromkeys(granule_names_clean)
granule_names = list(dico.keys())

In [120]:
len(granule_names)

63

In [124]:
granule_list_string = ','.join(granule_names)

In [164]:
base = 'https://api.daac.asf.alaska.edu/services/search/param'
payload = {
    'granule_list': granule_list_string,
    'processingLevel': 'INTERFEROMETRY_GRD',
    'output': 'JSON'
}
r = requests.get(base, params=payload)

result = r.json()[0]

In [165]:
len(result)

63

In [166]:
res_dict = {
    k: [d.get(k) for d in result]
    for k in set().union(*result)
}

In [167]:
df = pd.DataFrame.from_dict(res_dict)
df.head()

,processingDate,collectionName,sceneDateString,product_file_id,beamModeType,polarization,track,nearStartLon,nearEndLon,centerLon,...,sizeMB,browse,nearStartLat,farStartLon,sarSceneId,masterGranule,insarStackSize,sceneDate,processingLevel,doppler
0,2021-04-20T14:05:11Z,"Grand Mesa, CO",None,UA_grmesa_27416_21019-017_21021-005_0006d_s01_...,RPI,HH,27416,-108.29981431,-108.28850828,-108.110831843488,...,1061.31871414,https://datapool.asf.alaska.edu/BROWSE/UA/grme...,38.97978927,-107.92851476,None,None,0,2021-03-22T15:30:53Z,INTERFEROMETRY_GRD,-1
1,2021-04-16T00:20:18Z,"Grand Mesa, CO",None,UA_grmesa_27416_21017-017_21019-017_0006d_s01_...,RPI,HH,27416,-108.29748606,-108.29003968,-108.109932476343,...,1059.69205761,https://datapool.asf.alaska.edu/BROWSE/UA/grme...,38.97964027,-107.92584069,None,None,0,2021-03-16T16:39:43Z,INTERFEROMETRY_GRD,-1
2,2021-04-16T00:31:52Z,"Grand Mesa, CO",None,UA_grmesa_27416_21016-002_21017-017_0007d_s01_...,RPI,HH,27416,-108.36892775,-108.36503558,-108.13248652742,...,1395.20475578,https://datapool.asf.alaska.edu/BROWSE/UA/grme...,38.98395834,-107.89145644,None,None,0,2021-03-10T16:36:09Z,INTERFEROMETRY_GRD,-1
3,2021-04-16T00:11:26Z,"Grand Mesa, CO",None,UA_grmesa_27416_21011-010_21016-002_0021d_s01_...,RPI,HH,27416,-108.37162923,-108.36243788,-108.133431122101,...,1387.80714989,https://datapool.asf.alaska.edu/BROWSE/UA/grme...,38.98414439,-107.894959,None,None,0,2021-03-03T22:03:01Z,INTERFEROMETRY_GRD,-1
4,2021-04-15T00:39:39Z,"Utica, MT",None,UA_uticam_21003_21004-002_21013-003_0034d_s01_...,RPI,HH,21003,-110.53347332,-109.94382806,-110.121192870436,...,3580.7259922,https://datapool.asf.alaska.edu/BROWSE/UA/utic...,46.53446577,-110.30794956,None,None,0,2021-02-23T18:47:21Z,INTERFEROMETRY_GRD,-1


In [163]:
df.to_csv('interferometric_grd_metadatas.csv', index=False)